In [1]:
import os, re, json , glob

In [11]:
import re
from bs4 import BeautifulSoup

# Sample README content (you can load this from a file instead)
# readme_content = '''
# <details>
# <summary>Click the arrow to view options.</summary>

#   - m: metre (unit for length).
#   - Pa: pascal (unit for pressure).
#   - K: kelvin jslj.

# </details>
# '''

def extract_bullets_with_brackets(html_text):
    soup = BeautifulSoup(html_text, "html.parser")
    results = {}

    bullet_pattern = re.compile(r"\s*-\s*(\w+):\s*([^\(]+?)(?:\s*\((.*?)\))?\.")

    for details in soup.find_all("details"):
        lines = details.get_text().splitlines()
        for line in lines:
            match = bullet_pattern.match(line)
            if match:
                symbol, description, bracket_info = match.groups()
                results[symbol] = {
                    "text1": description.strip(),
                    "text2": bracket_info.strip() if bracket_info else None
                }

    return results

# # Run the function
# bullets = extract_bullets_with_brackets(readme_content)

# # Print neatly
# from pprint import pprint
# pprint(bullets)


In [18]:
rme = glob.glob('*/*README*')
rme

['native-horizontal-grid-type/README.md',
 'native-horizontal-grid-region/README.md',
 'model-calendar/README.md',
 'native-horizontal-grid-nominal-resolution/README.md',
 'model-component/README.md',
 'native-horizontal-grid-temporal-refinement/README.md',
 'native-horizontal-grid-grid-mapping/README.md',
 'native-horizontal-grid-truncation-method/README.md',
 'model-component-type/README.md',
 'native-vertical-grid-coordinate/README.md',
 'native-horizontal-grid-arrangement/README.md',
 'model/README.md',
 'reference/README.md',
 'native-vertical-grid-units/README.md',
 'grid/README.md']

In [ ]:
for rd in rme:
    path = os.path.dirname(rd.replace('README.md',''))
    print(path)
    mytype = path.split('/')[-1]
    
    context = {
    "@context": [
        "../_context_",
        {
            "@base": f"https://wcrp-cmip.github.io/WCRP-universe/{mytype}/",
            "@vocab": f"https://wcrp-cmip.github.io/WCRP-universe/{mytype}/"
        }
    ],
    "@embed": "@always",
    "@type": [
        "universal",
        f"wcrp:{mytype}",
    ]
    }
    with open(f'./{path}/_context_', 'w') as f:
        json.dump(context, f, indent=4)
    
    branches = extract_bullets_with_brackets(open(rd).read())
    # print(branches)
    
    for k, v in branches.items():
        validation = k.replace('_','-').replace(' ','-')
        filename = validation.lower()
        
        
        ld = {
            "id": filename,
            "type": [f"wcrp:{mytype}","universal"],
            "validation-key": validation,

            
        } 
        
        if v['text2'] is not None:
            
            ld['description'] =  v['text2']
            ld['ui-label'] = v['text1']
            
        else:
            ld['description'] = v['text1']
            ld['ui-label'] = validation.replace('-',' ').capitalize()
            
        print(ld)
        
        with open(f'./{path}/{filename}.json', 'w') as f:
            json.dump(ld, f, indent=4)

native-horizontal-grid-type
{'id': 'regular-latitude-longitude', 'type': ['wcrp:native-horizontal-grid-type', 'universal'], 'validation-key': 'regular-latitude-longitude', 'description': 'a rectilinear latitude-longitude grid with evenly spaced latitude points and evenly spaced longitude points', 'ui-label': 'Regular latitude longitude'}
{'id': 'regular-gaussian', 'type': ['wcrp:native-horizontal-grid-type', 'universal'], 'validation-key': 'regular-gaussian', 'description': 'a Gaussian grid for which the number of longitudinal points is constant for each latitude', 'ui-label': 'Regular gaussian'}
{'id': 'reduced-gaussian', 'type': ['wcrp:native-horizontal-grid-type', 'universal'], 'validation-key': 'reduced-gaussian', 'description': 'a Gaussian grid for which the number of longitudinal points is reduced as the poles are approached', 'ui-label': 'Reduced gaussian'}
{'id': 'spectral-gaussian', 'type': ['wcrp:native-horizontal-grid-type', 'universal'], 'validation-key': 'spectral-gaussian